## 상대모멘텀
- 10 개의 종목에서 전월의 흐름을 보고 어떤 주식에 투자할 것인가
1. 월초부터 월말까지의 수정종가를 이용하여 수익률 생성
2. 월별 수익률이 높은 n개
3. 해당 종목을 매수하고 있다가 해당 종목의 신호가 없어지면 매도하는 형식으로 수익률 생성

In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
def create_1m_rtn(
    _df,
    _ticker,
    _start = "2010-01-01",
    _col = "Adj Close"  
):
    result = _df.copy()
    
    if "Date" in result.columns:
        result.set_index("Date", inplace = True)
    
    result.index = pd.to_datetime(result.index)
    
    result = result.loc[_start:,[_col]]
    
    result['STD-YM'] = result.index.strftime("%Y-%m")
    result['1m_rtn'] = 0
    result['CODE'] = _ticker
    
    ym_list = result['STD-YM'].unique()
    return result, ym_list

In [4]:
df = pd.read_csv("../../data/csv/AAPL.csv", index_col = "Date")
sample_aapl, ym_list = create_1m_rtn(df, 'AAPL')

sample_aapl , ym_list

(             Adj Close   STD-YM  1m_rtn  CODE
 Date                                         
 2010-01-04   26.782711  2010-01       0  AAPL
 2010-01-05   26.829010  2010-01       0  AAPL
 2010-01-06   26.402260  2010-01       0  AAPL
 2010-01-07   26.353460  2010-01       0  AAPL
 2010-01-08   26.528664  2010-01       0  AAPL
 ...                ...      ...     ...   ...
 2019-06-18  198.449997  2019-06       0  AAPL
 2019-06-19  197.869995  2019-06       0  AAPL
 2019-06-20  199.460007  2019-06       0  AAPL
 2019-06-21  198.779999  2019-06       0  AAPL
 2019-06-24  199.169998  2019-06       0  AAPL
 
 [2384 rows x 4 columns],
 array(['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06',
        '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12',
        '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
        '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
        '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2

In [5]:
# os.listdir('./data/csv)
files = glob('./data/csv/*.csv')

In [6]:
stock_df = pd.DataFrame()
month_last_df = pd.DataFrame()

for file in files:
    file_path, file_name = os.path.split(file)
    head , tail = file_name.split(".")
    
    read_df = pd.read_csv(file_path + "/" + file_name)
    price_df , ym_list = create_1m_rtn(read_df, head)
    
    stock_df = pd.concat([stock_df, price_df])
    
    for ym in ym_list:
        flag = price_df['STD-YM'] == ym
        # 구매가(월초 수정종가)
        buy = price_df.loc[flag].iloc[0,0]
        # 판매가(월말 수정종가)
        sell = price_df.loc[flag].iloc[-1,0]
        m_rtn = buy / sell
        
        price_df.loc[flag, '1m_rtn'] = m_rtn
        
        last_data = price_df.loc[flag , ['CODE', '1m_rtn']].tail(1)
         
        month_last_df = pd.concat([month_last_df, last_data])
        

month_last_df
        

,CODE,1m_rtn
Date,,
2010-01-29,AAPL,1.114287
2010-02-26,AAPL,0.951666
2010-03-31,AAPL,0.889319
2010-04-30,AAPL,0.903788
2010-05-28,AAPL,1.036865
...,...,...
2019-02-28,USM,1.222960
2019-03-29,USM,1.025702
2019-04-30,USM,0.965281


In [7]:
month_rtn_df = month_last_df.copy()

month_rtn_df.reset_index(inplace=True)

In [8]:
month_rtn_df

,Date,CODE,1m_rtn
0,2010-01-29,AAPL,1.114287
1,2010-02-26,AAPL,0.951666
2,2010-03-31,AAPL,0.889319
3,2010-04-30,AAPL,0.903788
4,2010-05-28,AAPL,1.036865
...,...,...,...
1125,2019-02-28,USM,1.222960
1126,2019-03-29,USM,1.025702
1127,2019-04-30,USM,0.965281
1128,2019-05-31,USM,1.118485


### 피벗테이블

In [9]:
# 피벗테이블 재구조화

month_rtn_df = month_rtn_df.pivot_table(
    index = "Date",
    columns = "CODE",
    values = "1m_rtn"
)

In [10]:
month_rtn_df

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,1.114287,1.067698,0.988443,1.171660,1.036240,NaN,1.098297,1.083648,1.055313,1.148209
2010-02-26,0.951666,1.003970,0.996106,0.978355,0.990131,NaN,0.986319,1.018046,0.984829,0.995086
2010-03-31,0.889319,0.917287,1.003283,1.005179,1.004406,NaN,0.990782,0.939907,0.952403,0.894877
2010-04-30,0.903788,0.961415,0.989098,0.918630,0.955791,NaN,0.954813,0.960570,0.991499,1.003327
2010-05-28,1.036865,1.095887,0.988317,1.000401,0.973503,NaN,1.190751,1.021631,1.100393,1.032041
...,...,...,...,...,...,...,...,...,...,...
2019-02-28,0.957603,0.991706,0.998121,1.014838,1.004113,0.982270,0.913531,1.019822,0.969068,1.222960
2019-03-29,0.921137,0.938779,0.977950,0.967886,0.998934,1.055028,0.954129,1.001410,0.988354,1.025702
2019-04-30,0.953007,0.941693,0.996295,1.051221,1.002723,0.969448,0.911332,1.009272,0.972145,0.965281


In [11]:
# rank를 %로 표시

month_rtn_df = month_rtn_df.rank(
    axis=1,
    ascending=False,
    method='max',
    pct = True,
)

In [12]:
# 상위 15%만 출력

month_rtn_df = month_rtn_df.where(
    month_rtn_df < 0.15, 0
)

In [13]:
# 0과 1로 만들기

month_rtn_df[month_rtn_df != 0] = 1
month_rtn_df

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-02-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2010-03-31,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-04-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2010-05-28,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2019-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2019-03-29,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2019-04-30,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
slg_dict = dict()

for date in month_rtn_df.index:
    ticker_list = list(
        month_rtn_df.loc[date, month_rtn_df.loc[date] >= 1].index
    )
    slg_dict[date] = ticker_list
    
stock_codes = list(month_rtn_df.columns)

In [15]:
stock_df.head(2)

,Adj Close,STD-YM,1m_rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL


In [16]:
def create_trade_book(
    _df,
    _codes
):
    book = _df.copy()
    book['STD-YM'] = book.index.strftime('%Y-%m')
    for c in _codes:
        book[f"p_{c}"] = ''
        book[f"r_{c}"] = ''
        
    return book

In [17]:
stock_c_matrix = stock_df.reset_index().pivot_table(
    index = "Date",
    columns = "CODE",
    values = stock_df.columns[0]
)

In [18]:
stock_c_matrix

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-04,26.782711,133.899994,60.611969,44.908779,109.800003,NaN,24.525019,17.230000,93.675278,36.015179
2010-01-05,26.829010,134.690002,60.789135,45.341774,109.699997,NaN,24.532942,17.510000,93.923241,35.998024
2010-01-06,26.402260,132.250000,60.766037,46.443077,111.510002,NaN,24.382378,17.860001,93.989357,35.680672
2010-01-07,26.353460,130.000000,60.719822,46.217175,110.820000,NaN,24.128809,17.889999,94.386139,35.208931
2010-01-08,26.528664,133.520004,60.781410,46.913723,111.370003,NaN,24.295214,18.150000,94.700218,34.651424
...,...,...,...,...,...,...,...,...,...,...
2019-06-18,198.449997,1901.369995,82.397118,23.670000,127.120003,36.700001,135.160004,14.050000,290.984741,50.540001
2019-06-19,197.869995,1908.790039,82.676468,24.000000,127.889999,36.779999,135.690002,14.170000,291.641541,50.040001
2019-06-20,199.460007,1918.189941,82.806168,25.049999,131.110001,36.959999,136.949997,14.450000,294.427979,49.320000


In [19]:
book = create_trade_book(stock_c_matrix, stock_codes)


In [20]:
book.columns

Index(['AAPL', 'AMZN', 'BND', 'GDX', 'GLD', 'GM', 'MSFT', 'SLV', 'SPY', 'USM',
       'STD-YM', 'p_AAPL', 'r_AAPL', 'p_AMZN', 'r_AMZN', 'p_BND', 'r_BND',
       'p_GDX', 'r_GDX', 'p_GLD', 'r_GLD', 'p_GM', 'r_GM', 'p_MSFT', 'r_MSFT',
       'p_SLV', 'r_SLV', 'p_SPY', 'r_SPY', 'p_USM', 'r_USM'],
      dtype='object', name='CODE')

In [21]:
for date, values in slg_dict.items():

    for stock in values:
        book.loc[date, f"p_{stock}"] = f"ready_{stock}"
        
book['p_BND']

Date
2010-01-04             
2010-01-05             
2010-01-06             
2010-01-07             
2010-01-08             
                ...    
2019-06-18             
2019-06-19             
2019-06-20             
2019-06-21             
2019-06-24    ready_BND
Name: p_BND, Length: 2384, dtype: object

In [22]:
# 거래내역 추가
def trading(
    _book,
    _codes
):
    book = _book.copy()
    std_ym = ""
    buy_phase = False
    
    # 종목별
    for code in _codes:
        for idx in book.index:
            
            # 포지션
            if(book.loc[idx, f'p_{code}'] == "") & (book.shift().loc[idx, f'p_{code}'] == f"ready_{code}"):
                std_ym = book.loc[idx, 'STD-YM']
                buy_phase = True
            
            # 신호가 있으면 매수 상태 유지
            if(book.loc[idx, f"p_{code}"] == "") & (book.loc[idx, 'STD-YM'] == std_ym) & (buy_phase):
                book.loc[idx, f"p_{code}"] = f"buy_{code}"
                
            # 모두 만족할 때, std_ym, buy_phase 초기화
            if book.loc[idx, f"p_{code}"] == "":
                std_ym = ""
                buy_phase = False
                
    return book

In [23]:
book2 = trading(book, stock_codes)

In [24]:
book2['p_SPY'].value_counts()

p_SPY
    2384
Name: count, dtype: int64

In [25]:
# 수익률 계산 함수
def multi_returns(
    _book,
    s_code
):
    book = _book.copy()
    rtn = 1
    buy_dict = dict()
    sell_dict = dict()
    
    for i in book.index:
        for code in s_code:
            if(book.shift(2).loc[i, f"p_{code}"] == "") & (book.shift(1).loc[i, f"p_{code}"] == f"ready_{code}") & (book.loc[i, f"p_{code}"] == f"buy_{code}"):
                buy_dict[code] = book.loc[i, code]
                print(f"매수일 : {i}, 종목코드 : {code}, 매수가 : {buy_dict[code]}")
                
            elif (book.shift(1).loc[i, f"p_{code}"] == f"buy_{code}") & (book.loc[i, f"p_{code}"] == ""):
                sell_dict[code] = book.loc[i, code]
                # 수익률 계산
                rtn = sell_dict[code] / buy_dict[code]
                book.loc[i, f"r_{code}"] = rtn
                print(f"매도일 : {i}, 종목코드 : {code}, 매도가 : {sell_dict[code]}, 수익률 : {rtn}")
                
            if book.loc[i, f"p_{code}"] == "":
                buy_dict[code] = 0
                sell_dict[code] = 0
                    
    return book
                

In [26]:
book3 = multi_returns(book2, stock_codes)

매수일 : 2010-02-01 00:00:00, 종목코드 : GDX, 매수가 : 40.418842
매도일 : 2010-03-01 00:00:00, 종목코드 : GDX, 매도가 : 42.019035, 수익률 : 1.0395902732690858
매수일 : 2010-03-01 00:00:00, 종목코드 : SLV, 매수가 : 16.110001
매수일 : 2010-04-01 00:00:00, 종목코드 : GDX, 매수가 : 43.675705
매도일 : 2010-04-01 00:00:00, 종목코드 : SLV, 매도가 : 17.540001, 수익률 : 1.088764736886112
매도일 : 2010-05-03 00:00:00, 종목코드 : GDX, 매도가 : 46.951374, 수익률 : 1.074999796797785
매수일 : 2010-05-03 00:00:00, 종목코드 : USM, 매수가 : 36.744232
매수일 : 2010-06-01 00:00:00, 종목코드 : MSFT, 매수가 : 20.70437
매도일 : 2010-06-01 00:00:00, 종목코드 : USM, 매도가 : 35.003082, 수익률 : 0.9526143314139754
매수일 : 2010-07-01 00:00:00, 종목코드 : AMZN, 매수가 : 110.959999
매도일 : 2010-07-01 00:00:00, 종목코드 : MSFT, 매도가 : 18.521185, 수익률 : 0.8945543863445252
매도일 : 2010-08-02 00:00:00, 종목코드 : AMZN, 매도가 : 120.07, 수익률 : 1.0821016680074051
매수일 : 2010-08-02 00:00:00, 종목코드 : GDX, 매수가 : 45.228813
매도일 : 2010-09-01 00:00:00, 종목코드 : GDX, 매도가 : 49.746994, 수익률 : 1.0998960773080646
매수일 : 2010-09-01 00:00:00, 종목코드 : USM, 매수가 : 36.6

In [27]:
book3['p_SPY'].value_counts()

p_SPY
    2384
Name: count, dtype: int64

In [32]:
def multi_acc_returns(
    _book,
    s_code
):
    book = _book.copy()
    # 누적 수익률
    acc_rtn = 1
    for i in book.index:
        count = 0
        rtn = 0
        for code in s_code:
            if book.loc[i, f"r_{code}"]:
                count += 1
                rtn += book.loc[i, f"r_{code}"]
        if (rtn != 0) & (count != 0):
            acc_rtn *= (rtn / count)
            print(f"누적 매도일 : {i}, 매도 종목수 : {count}, 수익율 : {round(rtn / count, 2)}")
        
        book.loc[i, 'acc_rtn'] = acc_rtn

    return book, acc_rtn

In [33]:
multi_acc_returns(book3, stock_codes)

누적 매도일 : 2010-03-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.04
누적 매도일 : 2010-04-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.09
누적 매도일 : 2010-05-03 00:00:00, 매도 종목수 : 1, 수익율 : 1.07
누적 매도일 : 2010-06-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.95
누적 매도일 : 2010-07-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.89
누적 매도일 : 2010-08-02 00:00:00, 매도 종목수 : 1, 수익율 : 1.08
누적 매도일 : 2010-09-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.1
누적 매도일 : 2010-10-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.08
누적 매도일 : 2010-12-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.99
누적 매도일 : 2011-01-03 00:00:00, 매도 종목수 : 1, 수익율 : 1.07
누적 매도일 : 2011-02-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.0
누적 매도일 : 2011-03-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.1
누적 매도일 : 2011-05-02 00:00:00, 매도 종목수 : 1, 수익율 : 0.98
누적 매도일 : 2011-06-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.0
누적 매도일 : 2011-08-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.07
누적 매도일 : 2011-09-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.98
누적 매도일 : 2011-10-03 00:00:00, 매도 종목수 : 1, 수익율 : 0.86
누적 매도일 : 2011-11-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.1
누적 매도일 : 2011-12-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.9

(CODE              AAPL         AMZN        BND        GDX         GLD  \
 Date                                                                    
 2010-01-04   26.782711   133.899994  60.611969  44.908779  109.800003   
 2010-01-05   26.829010   134.690002  60.789135  45.341774  109.699997   
 2010-01-06   26.402260   132.250000  60.766037  46.443077  111.510002   
 2010-01-07   26.353460   130.000000  60.719822  46.217175  110.820000   
 2010-01-08   26.528664   133.520004  60.781410  46.913723  111.370003   
 ...                ...          ...        ...        ...         ...   
 2019-06-18  198.449997  1901.369995  82.397118  23.670000  127.120003   
 2019-06-19  197.869995  1908.790039  82.676468  24.000000  127.889999   
 2019-06-20  199.460007  1918.189941  82.806168  25.049999  131.110001   
 2019-06-21  198.779999  1911.300049  82.576698  25.209999  131.979996   
 2019-06-24  199.169998  1907.953857  82.726349  25.703501  133.501907   
 
 CODE               GM        MSFT  